# The CNN With Residual Blocks Model k-fold Cross Validation

CHEST X-RAY IMAGES CLASSIFICATION WITH CNN

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import os, shutil, pathlib


print("tensorflow version:", tf.__version__)
print("numpy version:", np.__version__)
print("pandas version:", pd.__version__)

tensorflow version: 2.10.0
numpy version: 1.23.4
pandas version: 1.5.1


## Paths of Data

In [2]:
DATA_PATH = "../data"
RAW_DATASET_NAME = "20210708_görüntüler"   # raw dataset folder name
SUB_FOLDERS_PATH = f"{DATA_PATH}/{RAW_DATASET_NAME}"  # sub class folders

SUB_FOLDERS_PATH

'../data/20210708_görüntüler'

In [3]:
os.listdir(DATA_PATH)

['20210708_görüntüler', 'images_1', 'manuel_differet_person']

In [4]:
labels = {"covid+ ac grafileri" : "covid",
         "infiltratif akciğer hastası grafileri" : "infiltiratif",
         "normal akciğer grafileri": "normal"}

labels

{'covid+ ac grafileri': 'covid',
 'infiltratif akciğer hastası grafileri': 'infiltiratif',
 'normal akciğer grafileri': 'normal'}

# Prepare Data

In [5]:
def dataframe_for_folders(DATA_PATH, DATASET_NAME, labels):
    """create dataframe from data folders that every folder has one class data
    
    It prepares dataframe to put in  TF flow_from_dataframe function
    
    Args:
        DATA_PATH : parent folder relative path of DATASET_NAME. It contains all of data files.
        DATASET_NAME : dataset name which wanted to use - path from DATA_PATH to classes folders
        labels: a dict that contains folder names according to classes
        
    Returns:
        A pandas dataframe that has relative path to the dataset_name of data and labels
    """
    CLASS_FOLDERS_PATHS = os.listdir(f"{DATA_PATH}/{DATASET_NAME}")
    
    df = pd.DataFrame(columns=["path", "label"])  # empty dataframe
    
    for i in CLASS_FOLDERS_PATHS:
        # list files - images
        list_of_files = os.listdir(f"{DATASET_NAME}/{i}")
        
        for k, name in enumerate(list_of_files):
            list_of_files[k] = f"{i}/"+name       # add parent folder

        df = pd.concat([df, pd.DataFrame(list_of_files, columns=["path"])], ignore_index=True)
        df.fillna(labels[i], inplace=True)  # label column is NaN, so we put labels every iteration. because every folder has one class        
    
#     df.reset_index(inplace = True)  
    
    return df

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   path    336 non-null    object
 1   label   336 non-null    object
dtypes: object(2)
memory usage: 5.4+ KB


# New CNN with Residual Blocks

In [9]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity

## Identity Block
`The identity block is a slightly different and more powerful version of the standard block used in ResNets, and corresponds to the case where the input activation (say 𝑎[𝑙]) has the same dimension as the output activation (say 𝑎[𝑙+2]).`

[Source: Convolutional Neural Networks on Coursera by Andrew NG](https://www.coursera.org/learn/convolutional-neural-networks)

<img src="images/idblock3_kiank.png" style="width:650px;height:150px;">
    <caption><center> <u> <font color='purple'> <b>Figure 1</b> </u><font color='purple'>  : <b>Identity block.</b> Skip connection "skips over" 3 layers.</center></caption>

In [10]:

def identity_block(X, f, filters, training=True, initializer=random_uniform):
    """
    Implementation of the identity block as defined in Figure 1
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to random uniform initializer
    
    Returns:
    X -- output of the identity block, tensor of shape (m, n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    X = Activation('relu')(X)
    
    ## Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = f, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)
    X = Activation('relu')(X)

    ## Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed = 0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) 
    
    ## Final step:
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)

    return X


## The Convolutional Block 
`The ResNet "convolutional block" is the second block type. You can use this type of block when the input and output dimensions don't match up.`

[Source: Convolutional Neural Networks on Coursera by Andrew NG](https://www.coursera.org/learn/convolutional-neural-networks)

<img src="images/convblock_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> <b>Figure 2</b> </u><font color='purple'>  : <b>Convolutional block</b> </center></caption>

In [11]:

def convolutional_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    s -- Integer, specifying the stride to be used
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to Glorot uniform initializer, 
                   also called Xavier uniform initializer.
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    
    # First component of main path glorot_uniform(seed=0)
    X = Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)
    
    ## Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = f, strides = (1, 1), padding = 'same', kernel_initializer = initializer(seed = 0))(X)
    X = BatchNormalization(axis = 3)(X, training=training) 
    X = Activation('relu')(X)

    ## Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed = 0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    
    ##### SHORTCUT PATH ##### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = 1, strides = (s,s), padding = 'valid', kernel_initializer = initializer(seed = 0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut, training=training)

    # Final step: Add shortcut value to main path (Use this order [X, X_shortcut]), and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

# CNN Model

In [12]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import StratifiedKFold

In [13]:
width = 480
height = 480

input_size = (width, height)
input_shape = (width, height,3)


batch_size = 4
val_batch_size = 4
test_batch_size = 4

epochs = 20

def ResNet50(input_shape = (64, 64, 3), classes = 6):
    """
    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = layers.Conv2D(64, 3, strides=2, padding="same", use_bias=False)(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 128], s = 1)
    X = identity_block(X, 3, [64, 64, 128])
    X = identity_block(X, 3, [64, 64, 128])
    
    #### NEW Layers
    for size in [32, 64, 128]:

        X = layers.BatchNormalization()(X)
        X = layers.Activation("relu")(X)

        X = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(X)
        X = layers.BatchNormalization()(X)
        X = layers.Activation("relu")(X)

        X = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(X)
        X = layers.MaxPooling2D(3, strides=2, padding="same")(X)
        
        X = layers.Conv2D(size, 3, strides=2, padding="same", use_bias=False)(X)
        X = layers.BatchNormalization()(X)


    ## AVGPOOL
    X = AveragePooling2D(pool_size = (2, 2))(X)


    # output layer
    X = Flatten()(X)
    
    X = layers.Dropout(0.5)(X)
    X = layers.Dense(64, activation="relu")(X)
    X = layers.Dropout(0.5)(X)
    
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X)
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [14]:
model = ResNet50(input_shape = input_shape, classes = 3)  # covid19 - infiltratrif - normal
print(input_shape)
print(model.summary())

(480, 480, 3)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 480, 480, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 486, 486, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 240, 240, 64  9472        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                

 add_1 (Add)                    (None, 120, 120, 12  0           ['activation_3[0][0]',           
                                8)                                'batch_normalization_8[0][0]']  
                                                                                                  
 activation_6 (Activation)      (None, 120, 120, 12  0           ['add_1[0][0]']                  
                                8)                                                                
                                                                                                  
 conv2d_9 (Conv2D)              (None, 120, 120, 64  8256        ['activation_6[0][0]']           
                                )                                                                 
                                                                                                  
 batch_normalization_9 (BatchNo  (None, 120, 120, 64  256        ['conv2d_9[0][0]']               
 rmalizati

                                                                                                  
 batch_normalization_18 (BatchN  (None, 8, 8, 64)    256         ['batch_normalization_17[0][0]'] 
 ormalization)                                                                                    
                                                                                                  
 activation_14 (Activation)     (None, 8, 8, 64)     0           ['batch_normalization_18[0][0]'] 
                                                                                                  
 separable_conv2d_4 (SeparableC  (None, 8, 8, 128)   8768        ['activation_14[0][0]']          
 onv2D)                                                                                           
                                                                                                  
 batch_normalization_19 (BatchN  (None, 8, 8, 128)   512         ['separable_conv2d_4[0][0]']     
 ormalizat

# K-fold Validation - 5 splits

In [15]:
SUB_FOLDERS_PATH

'../data/20210708_görüntüler'

In [16]:
# stratified k-fold splits data for a percentage of samples for each class
cv = StratifiedKFold(n_splits=5,shuffle=False)

for train_index, test_index in cv.split(df, df.label):
    print("length of train_index: ", len(train_index))
    print("length of test_index: ", len(test_index))
    print()

length of train_index:  268
length of test_index:  68

length of train_index:  269
length of test_index:  67

length of train_index:  269
length of test_index:  67

length of train_index:  269
length of test_index:  67

length of train_index:  269
length of test_index:  67



In [17]:
k = 0

all_train_scores = []   # all train scores on k-fold test data
all_val_scores = []   # all validation scores on k-fold test data


for train_index, test_index in cv.split(df, df.label):
    
    df_train = df.iloc[train_index]
    df_test = df.iloc[test_index]
    
    print("\n-----------------------------")
    print('Initializing Kfold %s'%str(k))
    print('Train shape:', df_train.shape)
    print('Test shape:', df_test.shape)
    

    
    # rescaling was not used because rescaling was used in model layers.    
    train_datagen = ImageDataGenerator(rescale=1./255)
    

    train_generator = train_datagen.flow_from_dataframe(dataframe=df_train,
                                                directory=SUB_FOLDERS_PATH,
                                                x_col="path",
                                                y_col="label",
                                                batch_size=batch_size,
                                                seed=42,
                                                shuffle=True,
                                                class_mode="categorical",
                                                target_size=input_size)

    validation_generator = train_datagen.flow_from_dataframe(dataframe=df_test,
                                                directory=SUB_FOLDERS_PATH,
                                                x_col="path",
                                                y_col="label",
                                                batch_size=val_batch_size,
                                                seed=42,
                                                shuffle=True,
                                                class_mode="categorical",
                                                target_size=input_size)
    
    model = ResNet50(input_shape = input_shape, classes = 3)
    model.fit(train_generator, epochs=epochs, batch_size=batch_size, verbose=0)
    
    train_generator.reset()
    train_loss, train_accuracy = model.evaluate(train_generator, verbose=0)
    print("train_loss: ", train_loss, "  train_accuracy", train_accuracy)
    all_train_scores.append(train_accuracy)
    
    validation_generator.reset()
    val_loss, val_accuracy = model.evaluate(validation_generator, verbose=0)
    print("val_loss: ", val_loss, "  val_accuracy: ", val_accuracy)
    all_val_scores.append(val_accuracy)
    
    k+=1
    


-----------------------------
Initializing Kfold 0
Train shape: (268, 2)
Test shape: (68, 2)
Found 268 validated image filenames belonging to 3 classes.
Found 68 validated image filenames belonging to 3 classes.
train_loss:  0.26112616062164307   train_accuracy 0.858208954334259
val_loss:  0.3500613868236542   val_accuracy:  0.779411792755127

-----------------------------
Initializing Kfold 1
Train shape: (269, 2)
Test shape: (67, 2)
Found 269 validated image filenames belonging to 3 classes.
Found 67 validated image filenames belonging to 3 classes.
train_loss:  0.2243489772081375   train_accuracy 0.9070631861686707
val_loss:  0.3747405409812927   val_accuracy:  0.8507462739944458

-----------------------------
Initializing Kfold 2
Train shape: (269, 2)
Test shape: (67, 2)
Found 269 validated image filenames belonging to 3 classes.
Found 67 validated image filenames belonging to 3 classes.
train_loss:  0.20278391242027283   train_accuracy 0.9033457040786743
val_loss:  0.483657538890

In [18]:
print("All train scores: ", all_train_scores)
print("\nMean of all train scores: ", np.mean(all_train_scores))
print("\nAll validation scores: ", all_val_scores)
print("\nMean of all validation scores: ", np.mean(all_val_scores))

All train scores:  [0.858208954334259, 0.9070631861686707, 0.9033457040786743, 0.8661710023880005, 0.8773234486579895]

Mean of all train scores:  0.8824224591255188

All validation scores:  [0.779411792755127, 0.8507462739944458, 0.8358209133148193, 0.8507462739944458, 0.9253731369972229]

Mean of all validation scores:  0.8484196782112121


In [19]:
print("All train scores: ", all_train_scores)
print("\nMean of all train scores: ", np.mean(all_train_scores))
print("\nAll validation scores: ", all_val_scores)
print("\nMean of all validation scores: ", np.mean(all_val_scores))

All train scores:  [0.858208954334259, 0.9070631861686707, 0.9033457040786743, 0.8661710023880005, 0.8773234486579895]

Mean of all train scores:  0.8824224591255188

All validation scores:  [0.779411792755127, 0.8507462739944458, 0.8358209133148193, 0.8507462739944458, 0.9253731369972229]

Mean of all validation scores:  0.8484196782112121
